# Importing necessary Libraries and Checking the Device specifications


In [ ]:
# --- Core Libraries ---
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torch.cuda.amp import GradScaler, autocast

# --- Analytics & Visualization ---
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from sklearn.metrics import classification_report, confusion_matrix
import random
import pickle
import warnings
warnings.filterwarnings('ignore')

# GPU Setup and Verification


In [ ]:
def setup_gpu():
    if torch.cuda.is_available():
        device_count = torch.cuda.device_count()
        current_device = torch.cuda.current_device()
        device_name = torch.cuda.get_device_name(current_device)
        
        print("="*60)
        print("TRANSFER LEARNING 2 - GPU CONFIGURATION")
        print("="*60)
        print(f"✅ CUDA Available: {torch.cuda.is_available()}")
        print(f"✅ PyTorch Version: {torch.__version__}")
        print(f"✅ GPU Count: {device_count}")
        print(f"✅ Current GPU: {current_device}")
        print(f"✅ GPU Name: {device_name}")
        
        gpu_memory = torch.cuda.get_device_properties(current_device).total_memory / 1e9
        print(f"✅ GPU Memory: {gpu_memory:.2f} GB")
        
        cudnn.benchmark = True
        cudnn.deterministic = True
        
        return f'cuda:{current_device}'
    else:
        print("❌ CUDA not available!")
        return 'cpu'

DEVICE = setup_gpu()
print("="*60)

# Importing the dataset

In [ ]:
# --- Reproducibility ---
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

# --- Configuration ---
DATA_DIR_VARIETY = '/kaggle/input/fruitvision-a-benchmark-dataset-for-fresh/Fruits Original'


BATCH_SIZE = 24  # Reduced for VGG16 which uses more memory
IMG_SIZE = (224, 224)
LEARNING_RATE = 0.0001
EPOCHS = 20
print(f"✅ Configuration: Batch={BATCH_SIZE}, Epochs={EPOCHS}, LR={LEARNING_RATE}")

# --- GPU Memory Management ---
def clear_gpu_memory():
    if DEVICE.startswith('cuda'):
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats(DEVICE)

def print_gpu_memory():
    if DEVICE.startswith('cuda'):
        allocated = torch.cuda.memory_allocated(DEVICE) / 1e9
        cached = torch.cuda.memory_reserved(DEVICE) / 1e9
        max_alloc = torch.cuda.max_memory_allocated(DEVICE) / 1e9
        print(f"GPU Memory - Allocated: {allocated:.2f}GB, Cached: {cached:.2f}GB, Max: {max_alloc:.2f}GB")

# Data Transformers and DataLoaders 

In [ ]:
train_transforms = transforms.Compose([
    transforms.TrivialAugmentWide(),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_test_transforms = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# --- GPU-Optimized DataLoader ---
def create_dataloaders(dataset_path, batch_size=24):
    full_dataset = datasets.ImageFolder(dataset_path)
    
    total_size = len(full_dataset)
    train_size = int(0.7 * total_size)
    val_size = int(0.15 * total_size)
    test_size = total_size - train_size - val_size
    
    train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size])
    
    train_dataset.dataset.transform = train_transforms
    val_dataset.dataset.transform = val_test_transforms
    test_dataset.dataset.transform = val_test_transforms
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                             num_workers=4, pin_memory=True, persistent_workers=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                           num_workers=4, pin_memory=True, persistent_workers=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                            num_workers=4, pin_memory=True, persistent_workers=True)
    
    return train_loader, val_loader, test_loader, train_size, val_size, test_size


# Transfer Learning Model Function

In [ ]:
# --- Transfer Learning Model Creation ---
def create_transfer_model(model_name, num_classes, freeze_backbone=True):
    print(f"\n--- Creating {model_name} ---")
    
    if model_name == "vgg16":
        model = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
        
        if freeze_backbone:
            for param in model.features.parameters():
                param.requires_grad = False
            print(f"✅ Backbone frozen for {model_name}")
        
        # Replace classifier with more efficient one
        model.classifier = nn.Sequential(
            nn.Linear(25088, 4096),
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(4096, 1024),
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.ReLU(True),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )
        
    elif model_name == "convnext_tiny":
        model = models.convnext_tiny(weights=models.ConvNeXt_Tiny_Weights.IMAGENET1K_V1)
        
        if freeze_backbone:
            for param in model.features.parameters():
                param.requires_grad = False
            print(f"✅ Backbone frozen for {model_name}")
        
        # Replace classifier
        model.classifier = nn.Sequential(
            nn.Flatten(1),
            nn.LayerNorm((768,), eps=1e-06, elementwise_affine=True),
            nn.Dropout(0.2),
            nn.Linear(768, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            nn.Linear(256, num_classes)
        )
    
    else:
        raise ValueError(f"Model {model_name} not supported")
    
    # Count parameters
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"✅ Total params: {total_params:,}")
    print(f"✅ Trainable params: {trainable_params:,}")
    
    return model.to(DEVICE)

# Function Declaration

In [ ]:
# --- Training Function ---
def train_transfer_model(model, model_name, train_loader, val_loader, epochs, lr, device, task_type):
    print(f"\n--- Training {model_name} for {task_type} ---")
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), 
                           lr=lr, weight_decay=1e-4)
    scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=5, eta_min=1e-7)
    scaler = GradScaler()
    
    # Early stopping
    early_stopping_patience = 7
    min_val_loss = float('inf')
    epochs_no_improve = 0
    best_model_state = None
    
    history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}
    
    clear_gpu_memory()
    
    for epoch in range(epochs):
        # Training phase
        model.train()
        train_loss, train_correct, train_total = 0, 0, 0
        
        for batch_idx, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            
            optimizer.zero_grad()
            
            with autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            
            # Scale loss for mixed precision
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            train_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
            
            # Memory management for VGG16
            if batch_idx % 50 == 0:
                torch.cuda.empty_cache()
        
        # Validation phase
        model.eval()
        val_loss, val_correct, val_total = 0, 0, 0
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
                
                with autocast():
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        
        # Calculate metrics
        avg_train_loss = train_loss / train_total
        avg_val_loss = val_loss / val_total
        avg_train_acc = train_correct / train_total
        avg_val_acc = val_correct / val_total
        
        history['train_loss'].append(avg_train_loss)
        history['val_loss'].append(avg_val_loss)
        history['train_acc'].append(avg_train_acc)
        history['val_acc'].append(avg_val_acc)
        
        # Print progress
        current_lr = optimizer.param_groups[0]['lr']
        print(f"Epoch {epoch+1}/{epochs} | Train Acc: {avg_train_acc:.4f} | Val Acc: {avg_val_acc:.4f} | "
              f"Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | LR: {current_lr:.6f}")
        
        scheduler.step()
        
        # Early stopping and model saving
        if avg_val_loss < min_val_loss:
            min_val_loss = avg_val_loss
            epochs_no_improve = 0
            best_model_state = model.state_dict().copy()
            torch.save(model.state_dict(), f'{model_name}_{task_type.lower().replace(" ", "_")}_best.pth')
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= early_stopping_patience:
                print(f"Early stopping at epoch {epoch+1}")
                break
        
        if epoch % 5 == 0:
            print_gpu_memory()
    
    # Load best model
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
    
    clear_gpu_memory()
    return history

# Model Evaluation and Metrics Defining Functions

In [ ]:
# --- Evaluation Function ---
def evaluate_transfer_model(model, model_name, test_loader, class_names, device, task_type):
    print(f"\n--- Evaluating {model_name} for {task_type} ---")
    model.eval()
    y_pred, y_true = [], []
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device, non_blocking=True)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            y_pred.extend(predicted.cpu().numpy())
            y_true.extend(labels.cpu().numpy())
    
    # Calculate metrics
    report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True, zero_division=0)
    accuracy = report['accuracy']
    f1_macro = report['macro avg']['f1-score']
    
    print(f"✅ Test Accuracy: {accuracy:.4f}")
    print(f"✅ F1-Macro Score: {f1_macro:.4f}")
    
    # Detailed report
    print("\nDetailed Classification Report:")
    print(classification_report(y_true, y_pred, target_names=class_names, zero_division=0))
    
    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', 
                xticklabels=class_names, yticklabels=class_names)
    plt.title(f'Confusion Matrix: {model_name} ({task_type})', fontsize=14)
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.xticks(rotation=45)
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()
    
    return {'accuracy': accuracy, 'f1_macro': f1_macro}

## VGG16 Training

In [ ]:
print("\n" + "="*60)
print("FRUITVISION")
print("="*60)

# Load class information
class_names_variety = sorted(os.listdir(DATA_DIR_VARIETY))
NUM_CLASSES_VARIETY = len(class_names_variety)
print(f"Variety Classes ({NUM_CLASSES_VARIETY}): {class_names_variety}")

# Create data loaders
train_loader_v, val_loader_v, test_loader_v, train_size_v, val_size_v, test_size_v = create_dataloaders(
    DATA_DIR_VARIETY, BATCH_SIZE
)
print(f"Dataset sizes: Train={train_size_v}, Val={val_size_v}, Test={test_size_v}")

# === VGG16 FOR VARIETY CLASSIFICATION ===
print("\n" + "="*40)
print("VGG16 - VARIETY CLASSIFICATION")
print("="*40)

vgg16_variety = create_transfer_model("vgg16", NUM_CLASSES_VARIETY, freeze_backbone=True)
history_vgg_variety = train_transfer_model(vgg16_variety, "VGG16", 
                                          train_loader_v, val_loader_v, EPOCHS, 
                                          LEARNING_RATE, DEVICE, "Variety Classification")

## VGG16 Evaluation

In [ ]:
# Plot training curves
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.plot(history_vgg_variety['train_acc'], label='Train', marker='o')
plt.plot(history_vgg_variety['val_acc'], label='Validation', marker='s')
plt.title('VGG16 Variety - Accuracy', fontsize=14)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(history_vgg_variety['train_loss'], label='Train', marker='o')
plt.plot(history_vgg_variety['val_loss'], label='Validation', marker='s')
plt.title('VGG16 Variety - Loss', fontsize=14)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Evaluate VGG16
results_vgg_variety = evaluate_transfer_model(vgg16_variety, "VGG16", 
                                             test_loader_v, class_names_variety, 
                                             DEVICE, "Variety Classification")

# Save model info
model_info_vgg_variety = {
    'model_name': 'VGG16_Variety',
    'architecture': 'VGG16 (Transfer Learning)',
    'num_classes': NUM_CLASSES_VARIETY,
    'class_names': class_names_variety,
    'input_size': IMG_SIZE,
    'accuracy': results_vgg_variety['accuracy'],
    'f1_macro': results_vgg_variety['f1_macro'],
    'frozen_backbone': True,
    'epochs_trained': len(history_vgg_variety['train_acc'])
}

with open('vgg16_variety_info.pkl', 'wb') as f:
    pickle.dump(model_info_vgg_variety, f)

clear_gpu_memory()

## ConvNeXt Training and evaluation

In [ ]:
print("\n" + "="*40)
print("CONVNEXT-TINY - VARIETY CLASSIFICATION")
print("="*40)

convnext_variety = create_transfer_model("convnext_tiny", NUM_CLASSES_VARIETY, freeze_backbone=True)
history_convnext_variety = train_transfer_model(convnext_variety, "ConvNeXt-Tiny",
                                               train_loader_v, val_loader_v, EPOCHS,
                                               LEARNING_RATE, DEVICE, "Variety Classification")

# Plot training curves
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.plot(history_convnext_variety['train_acc'], label='Train', marker='o')
plt.plot(history_convnext_variety['val_acc'], label='Validation', marker='s')
plt.title('ConvNeXt-Tiny Variety - Accuracy', fontsize=14)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(history_convnext_variety['train_loss'], label='Train', marker='o')
plt.plot(history_convnext_variety['val_loss'], label='Validation', marker='s')
plt.title('ConvNeXt-Tiny Variety - Loss', fontsize=14)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Evaluate ConvNeXt-Tiny
results_convnext_variety = evaluate_transfer_model(convnext_variety, "ConvNeXt-Tiny",
                                                  test_loader_v, class_names_variety,
                                                  DEVICE, "Variety Classification")

# Save model info
model_info_convnext_variety = {
    'model_name': 'ConvNeXt-Tiny_Variety',
    'architecture': 'ConvNeXt-Tiny (Transfer Learning)',
    'num_classes': NUM_CLASSES_VARIETY,
    'class_names': class_names_variety,
    'input_size': IMG_SIZE,
    'accuracy': results_convnext_variety['accuracy'],
    'f1_macro': results_convnext_variety['f1_macro'],
    'frozen_backbone': True,
    'epochs_trained': len(history_convnext_variety['train_acc'])
}

with open('convnext_tiny_variety_info.pkl', 'wb') as f:
    pickle.dump(model_info_convnext_variety, f)

clear_gpu_memory()

In [ ]:
# === COMPARISON TABLE ===
print("\n" + "="*60)
print("TRANSFER LEARNING MODELS COMPARISON")
print("="*60)

# Create comparison dataframe
comparison_data = [
    {
        'Model': 'VGG16',
        'Task': 'Variety Classification',
        'Accuracy': results_vgg_variety['accuracy'],
        'F1-Macro': results_vgg_variety['f1_macro'],
        'Epochs': len(history_vgg_variety['train_acc'])
    },
    {
        'Model': 'ConvNeXt-Tiny',
        'Task': 'Variety Classification', 
        'Accuracy': results_convnext_variety['accuracy'],
        'F1-Macro': results_convnext_variety['f1_macro'],
        'Epochs': len(history_convnext_variety['train_acc'])
    },
    {
        'Model': 'VGG16',
        'Task': 'Ripeness Detection',
        'Accuracy': results_vgg_ripeness['accuracy'],
        'F1-Macro': results_vgg_ripeness['f1_macro'],
        'Epochs': len(history_vgg_ripeness['train_acc'])
    },
    {
        'Model': 'ConvNeXt-Tiny',
        'Task': 'Ripeness Detection',
        'Accuracy': results_convnext_ripeness['accuracy'],
        'F1-Macro': results_convnext_ripeness['f1_macro'],
        'Epochs': len(history_convnext_ripeness['train_acc'])
    }
]

comparison_df = pd.DataFrame(comparison_data)
print(comparison_df.to_string(index=False, float_format='%.4f'))

# Visualization of results
plt.figure(figsize=(15, 6))

plt.subplot(1, 2, 1)
variety_models = ['VGG16', 'ConvNeXt-Tiny']
variety_acc = [results_vgg_variety['accuracy'], results_convnext_variety['accuracy']]
plt.bar(variety_models, variety_acc, color=['purple', 'gold'])
plt.title('Variety Classification - Test Accuracy')
plt.ylabel('Accuracy')
plt.ylim(0, 1)
for i, v in enumerate(variety_acc):
    plt.text(i, v + 0.01, f'{v:.3f}', ha='center')

plt.subplot(1, 2, 2)
ripeness_models = ['VGG16', 'ConvNeXt-Tiny']
ripeness_acc = [results_vgg_ripeness['accuracy'], results_convnext_ripeness['accuracy']]
plt.bar(ripeness_models, ripeness_acc, color=['darkgreen', 'crimson'])
plt.title('Ripeness Detection - Test Accuracy')
plt.ylabel('Accuracy')
plt.ylim(0, 1)
for i, v in enumerate(ripeness_acc):
    plt.text(i, v + 0.01, f'{v:.3f}', ha='center')

plt.tight_layout()
plt.show()

print("\n✅ All transfer learning models 2 trained and evaluated successfully!")
print("✅ Model weights and metadata saved")
print("✅ GPU memory managed efficiently for VGG16")

clear_gpu_memory()